# Build a basic model by extracting features via Inception v3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications.inception_v3 import InceptionV3
#from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
#from keras.layers import GlobalAveragePooling2D

# #Default InceptionV3 options
# keras.applications.inception_v3.InceptionV3(include_top=True
#                                             , weights='imagenet'
#                                             , input_tensor=None
#                                             , input_shape=None
#                                             , pooling=None
#                                             , classes=1000)

Using TensorFlow backend.


In [11]:
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
#Use pretrained ImageNet weights, leaving out the top layer, and compress the output to
#a 2-tensor using Global Average Pooling
inceptionv3 = InceptionV3(include_top=False, weights='imagenet', pooling='avg')

In [10]:
inceptionv3.layers_by_depth[0]

In [13]:
inceptionv3.input.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(3)])

In [14]:
inceptionv3.output.shape

TensorShape([Dimension(None), Dimension(2048)])

### For all image files in the listed directories, store the following in a dataframe:

* hash of the image (as a unique id)
* filename relative to the species directory (could also use this as unique id, hopefully)
* species (which should equal the directory in which the image is located)
* Once the above information is stored, pass each image through the Inception V3 feature extractor to get a list of 2048 features in a numpy array. Add 2048 colums to the dataframe for these features.

In [12]:
base_directory = 'tree_photos'
subdirectories = ['picea_sitchensis', 'platanus_acerifolia', 'acer_macrophylum']
image_extensions = ['png', 'jpg', 'jpeg']